In [18]:
# !pip install opencv-python
# !pip install tensorflow
# !pip install matplotlib

import os
import cv2
# import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import load_model


In [19]:
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 90
CLASSES_LIST = ["normalHead", "oddHead"]

In [20]:
filepath = "./LRCN_model___Date_Time_2021_12_10__13_58_06___Loss_1.2796132564544678___Accuracy_0.7021276354789734.h5"
##loading weights with Loss of 1.2796132564544678 and testing accuracy of 0.7021276354789734
LRCN_model=load_model(filepath)

In [21]:
def predict(path, SEQUENCE_LENGTH):
    video = cv2.VideoCapture(path)

    framesList = []
    
    predClass = ''

    lenFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if(lenFrames<SEQUENCE_LENGTH):
        return 0;
    print(path)

    framesToSkip = max(int(lenFrames/SEQUENCE_LENGTH),1)

    for i in range(SEQUENCE_LENGTH):
        video.set(cv2.CAP_PROP_POS_FRAMES, i * framesToSkip)
        success, frame = video.read() 

        if not success:
            break

        resizedFrame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
#         plt.imshow(frame)
#         plt.show()
        
        normalizedFrame = resizedFrame / 255
        framesList.append(normalizedFrame)

    predictedLabelsProbabilities = LRCN_model.predict(np.expand_dims(framesList, axis = 0))[0]

    predLabel = np.argmax(predictedLabelsProbabilities)

    predClass = CLASSES_LIST[predLabel]
    # Display the predicted action along with the prediction confidence.
    print(f'Action Predicted: {predClass}\nConfidence: {predictedLabelsProbabilities[predLabel]}')
        
    video.release()
    
    return predClass

In [22]:
classnameA=["oddHead","normalHead"]

total=0
sum=0

for classname in classnameA:
    path=f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{classname}Test'
    videosPath=os.listdir(path)

    for video in videosPath:
        inputVideoPath=os.path.join(f'{path}\\{video}')
        pred=predict(inputVideoPath, 90)
        if(pred==classname):
            sum+=1
        if(pred!=0):
            total+=1
            acc=sum/total
            print("Accuracy: ", acc)
        

print("DONE")

C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_12.mp4
Action Predicted: oddHead
Confidence: 0.9999797344207764
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_4.mp4
Action Predicted: oddHead
Confidence: 0.9999608993530273
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_6.mp4
Action Predicted: oddHead
Confidence: 0.9981369972229004
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_12_C_9.mp4
Action Predicted: oddHead
Confidence: 0.9999700784683228
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_1_C_2.mp4
Action Predicted: oddHead
Confidence: 0.9999866485595703
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_1_C_5.mp4
Action Predicted: oddHead
Confidence: 0.9999042749404907
Accuracy:  1.0
C:\Users\CSE-P07-2179\Documents\headmovement\oddHeadTest\HM_1_C_6.mp4
Action Predicted: normalHead
Confidence: 0.9999984502792358
Accuracy:  0.

In [5]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''
    # Declare a list to store video frames.
    frames_list = []
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    print(video_frames_count)
    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):
        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        # Reading the frame from the video.
        success, frame = video_reader.read()
        # Check if Video frame is not successfully read then break the loop
        if not success:
            break
        # Resize the Frame to fixed height and width.

#         plt.imshow(frame)
#         plt.show()

        # cropped_image = frame[350:850,650:2]

        # plt.imshow(cropped_image)
        # plt.show()

        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # plt.imshow(resized_frame)
        # plt.show()

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    # Release the VideoCapture object.
    video_reader.release()
    # Return the frames list.
    return frames_list

In [18]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''
    # Declared Empty Lists to store the features, labels and video file path values.
    features_train = []
    features_test = []

    labels_train = []
    labels_test = []

    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}')
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}\\{file_name}')
            print(video_file_path)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                # Append the data to their repective lists.
                features_train.append(frames)
                labels_train.append(class_index)
                video_files_paths.append(video_file_path)


    for class_index, class_name in enumerate(CLASSES_LIST):
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}Test')
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{class_name}Test\\{file_name}')
            print(video_file_path)
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                # Append the data to their repective lists.
                features_test.append(frames)
                labels_test.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features_train = np.asarray(features_train)
    labels_train = np.array(labels_train)

    features_test = np.asarray(features_test)
    labels_test = np.array(labels_test)

    # Return the frames, class index, and video file path.
    return features_train,features_test, labels_train,labels_test, video_files_paths

In [ ]:
features_train, features_test, labels_train, labels_test, video_files_paths = create_dataset()
print(np.shape(features_train))
print(np.shape(features_test))

labels_train = to_categorical(labels_train)
labels_test = to_categorical(labels_test)

Extracting Data of Class: normalHead
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_1.mp4
64
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_2.mp4
81
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_3.mp4
59
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_4.mp4
41
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_10_N_5.mp4
160
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_1.mp4
41
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_10.mp4
88
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_11.mp4
83
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_2.mp4
84
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_3.mp4
100
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_4.mp4
32
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_5.mp4
56
C:\Users\CSE-P07-2179\Documents\headmovement\normalHead\HM_13_N_6.mp4
71
C:\Users\C

In [19]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()

    # Define the Model Architecture.
    ########################################################################################################################

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'),
                              input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
#     model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
#     model.add(TimeDistributed(MaxPooling2D((2, 2))))
#     model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    # model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Flatten()))

    model.add(LSTM(64))

    model.add(Dense(len(CLASSES_LIST), activation='softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()

    # Return the constructed LRCN model.
    return model

In [20]:
LRCN_model = create_LRCN_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 45, 64, 64, 16)   448       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 45, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 45, 16, 16, 16)   0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 45, 16, 16, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 45, 4, 4, 32)     0

In [21]:
LRCN_model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=["accuracy"])

LRCN_model_training_history = LRCN_model.fit(x=features_train, y=labels_train, epochs=100, batch_size=4, shuffle=True,
                                             validation_split=0.2)

model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

NameError: name 'features_train' is not defined

In [55]:
import re
boundedfolder = "C:\\Users\\CSE-P07-2179\\Desktop\\Thesis\\headMovementDataset\\oddHead\\boundedFrames"

frames =[]

# path1 =os.path.join(boundedfolder,f"Bounded_Frame_HM_10_C_1_1.jpg")
# print(path1)
# print(os.path.isfile(path1))
for i in range(1,130):
       #print((os.path.join(boundedfolder,f"HM_10_C_1_1.jpg")))
        #frame=cv2.imread(os.path.join(boundedfolder,f"Bounded_Frame_HM_10_C_1_{i}.jpg") )
        #print(frame)
        frames.append(cv2.imread(os.path.join(boundedfolder,f"Bounded_Frame_HM_10_C_3_{i}.jpg") ))
        
        
print(len(frames))

129


In [ ]:
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_file_name = f'./LRCN_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

LRCN_model.save(model_file_name)

model_evaluation_accuracy

In [6]:
filepath = "./LRCN_model___Date_Time_2021_12_10__13_58_06___Loss_1.2796132564544678___Accuracy_0.7021276354789734.h5"

LRCN_model=load_model(filepath)


In [7]:
def predict(path, SEQUENCE_LENGTH):
    video = cv2.VideoCapture(path)

    framesList = []
    
    predClass = ''

    lenFrames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if(lenFrames<SEQUENCE_LENGTH):
        return 0;
    print(path)

    framesToSkip = max(int(lenFrames/SEQUENCE_LENGTH),1)

    for i in range(SEQUENCE_LENGTH):
        video.set(cv2.CAP_PROP_POS_FRAMES, i * framesToSkip)
        success, frame = video.read() 

        if not success:
            break

        resizedFrame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
#         plt.imshow(frame)
#         plt.show()
        
        normalizedFrame = resizedFrame / 255
        framesList.append(normalizedFrame)

    predictedLabelsProbabilities = LRCN_model.predict(np.expand_dims(framesList, axis = 0))[0]

    predLabel = np.argmax(predictedLabelsProbabilities)

    predClass = CLASSES_LIST[predLabel]
    # Display the predicted action along with the prediction confidence.
    print(f'Action Predicted: {predClass}\nConfidence: {predictedLabelsProbabilities[predLabel]}')
        
    video.release()
    
    return predClass

In [8]:
classnameA=["normalHead","oddHead"]

total=0
sum=0

for classname in classnameA:
    path=f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{classname}Test'
    videosPath=os.listdir(path)

    for video in videosPath:
        inputVideoPath=os.path.join(f'{path}\\{video}')
        pred=predict(inputVideoPath, 90)
        if(pred==classname):
            sum+=1
        if(pred!=0):
            total+=1
            acc=sum/total
            print("Accuracy: ", acc)
        

print("DONE")


C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_12_N_2.mp4
Action Predicted: oddHead
Confidence: 0.9819273352622986
Accuracy:  0.0
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_12_N_5.mp4
Action Predicted: oddHead
Confidence: 0.8815009593963623
Accuracy:  0.0
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_1_N_2.mp4
Action Predicted: normalHead
Confidence: 0.9999942779541016
Accuracy:  0.3333333333333333
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_1_N_7.mp4
Action Predicted: normalHead
Confidence: 0.9999974966049194
Accuracy:  0.5
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_21_N_1.mp4
Action Predicted: normalHead
Confidence: 0.9999663829803467
Accuracy:  0.6
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_21_N_2.mp4
Action Predicted: normalHead
Confidence: 0.8471510410308838
Accuracy:  0.6666666666666666
C:\Users\CSE-P07-2179\Documents\headmovement\normalHeadTest\HM_21_N_4.mp4
Action Pre

ValueError: in user code:

    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\CSE-P07-2179\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 263, in assert_input_compatibility
    

In [ ]:
classnameA=["normalHead","oddHead"]

total=0
sum=0

for classname in classnameA:
    path=f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\{classname}Test'
    videosPath=os.listdir(path)

    for video in videosPath:
        inputVideoPath=os.path.join(f'{path}\\{video}')

        pred=predict(inputVideoPath, 90)
        if(pred==classname):
            sum+=1
        if(pred!=0):
            total+=1
            acc=sum/total
            print(acc)
        

# boundedfolder
print("DONE")
            


In [ ]:
def predict_single_action2(video_reader, SEQUENCE_LENGTH):

    # Initialize the VideoCapture object to read from the video file.
    # video_reader = cv2.VideoCapture(video_reader)
    # Get the width and height of the video.
    # original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    # original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("here")
    frames_list = []

    predicted_class_name = ''

    # Get the number of frames in the video.
    # video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    video_frames_count = len(video_reader)
    if (video_frames_count < SEQUENCE_LENGTH - 1):
        print(video_frames_count)
        return 0
    # print(video_file_path)

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        # video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        frame=video_reader[frame_counter+skip_frames_window]



        # Read a frame.
        # success, frame = video_reader.read()

        # Check if frame is not read properly then break the loop.
        # if not success:
        #     break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        # plt.imshow(frame)
        # plt.show()

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_list, axis=0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]

    # Display the predicted action along with the prediction confidence.
    print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')

    # Release the VideoCapture object.

    return predicted_class_name

# predict_single_action(f'C:\\Users\\CSE-P07-2179\\Documents\\headmovement\\oddHeadTest\\HM_12_C_12.mp4', 90)   
x= predict_single_action2(frames, 90)   
print(x)